In [21]:
import pandas as pd
import numpy as np
import torch as pt
import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

#pt.set_default_tensor_type('torch.FloatTensor')
pt.set_default_tensor_type('torch.cuda.FloatTensor')

In [22]:
dataset = pd.read_csv(
    'data_files/kc_house_data.csv',
    index_col = 'id', parse_dates = ['date'], date_parser = lambda x: pd.datetime.strptime(x, '%Y%m%dT%H%M%S')
)
dataset = dataset[[col for col in dataset.columns if not col.endswith('15')]]
feature_cols = ['sqft_living', 'bedrooms', 'bathrooms']
target_cols = ['price']
dataset[target_cols] = np.log(dataset[target_cols])

In [27]:
alpha = 0.1
beta = 0.001
clamp = 1e-16
grad_smooth = 0
grad_sq_smooth = 0
learn_rate = 1e-2

feature_tensor = pt.from_numpy(dataset[feature_cols].values).type(pt.Tensor)
target_tensor = pt.from_numpy(dataset[target_cols].values).type(pt.Tensor)
coefs = pt.zeros(len(feature_cols) + 1, len(target_cols))

In [31]:
t = tqdm.tnrange(10000)
loss_rcd = []
for i in t:
    coefs.requires_grad = True
    predict_tensor = (feature_tensor @ coefs[1:, :]) + coefs[0, :]
    error = (target_tensor - predict_tensor) ** 2
    loss = pt.sqrt(error.mean(dim = 0)).sum()
    
    loss_rcd.append(loss.detach().cpu().numpy())
    t.set_postfix({'loss': loss_rcd[-1]})
    
    grad = pt.autograd.grad(loss, coefs)[0]
    with pt.no_grad():
        grad_smooth = (alpha * grad) + ((1 - alpha) * grad_smooth)
        grad_sq_smooth = (beta * (grad ** 2)) + ((1 - beta) * grad_sq_smooth)

        alpha_scaler = 1 - (alpha ** (i+1))
        beta_scaler = 1 - (beta ** (i+1))
        
        learn_step = (
            grad_smooth / alpha_scaler
        ) / pt.clamp(
            pt.sqrt(grad_sq_smooth / beta_scaler),
        clamp, np.inf)
        
        coefs = coefs - (learn_step * (learn_rate / grad.size()[0]))

In [45]:
predict_tensor = (feature_tensor @ coefs[1:, :]) + coefs[0, :]
error = (target_tensor - predict_tensor) ** 2
loss_base = pt.sqrt(error.mean(dim = 0)).sum()
loss_base

tensor(0.4046)

In [56]:
feature_import = []
for j in tqdm.tnrange(feature_tensor.size()[1]):
    loss_perms = []
    for i in tqdm.tnrange(10000, leave = False):
        perm_tensor = []
        permute = 0
        for i in range(feature_tensor.size()[1]):
            if i == permute:
                perm_tensor.append(feature_tensor[pt.randperm(feature_tensor.size()[0]), i])
            else:
                perm_tensor.append(feature_tensor[:, i])
        perm_tensor = pt.stack(perm_tensor, dim = 1)

        predict_perm = (perm_tensor @ coefs[1:, :]) + coefs[0, :]
        error_perm = (target_tensor - predict_perm) ** 2
        loss_perms.append(pt.sqrt(error_perm.mean(dim = 0)).sum().detach().cpu().numpy())
    feature_import.append(np.stack(loss_perms))

In [64]:
[(feat_loss.mean(), feat_loss.std()) for feat_loss in feature_import]

[(0.6874954, 0.0021893294),
 (0.68746775, 0.0022249473),
 (0.6874756, 0.0021876968)]